In [1]:
import tensorflow as tf
# import tensorflow as tf
print(tf.__version__)
import torch
torch.cuda.is_available()

2.3.1


True

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pickle
torch.__version__

Using TensorFlow backend.


'1.7.0+cu110'

In [3]:
import re
import nltk
import pickle

In [4]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'2.6.0'

In [5]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'2.6.0'

In [6]:
model = torch.load('./model.pth')

In [9]:
txt = open('./testdata/test(2030).txt', 'r').read()
txt = txt.split('\n\n')
test_sentences = []
for u in txt:
    sts = u.split('\n')
    test_sentences.append(sts[0])

In [8]:
with open('./data/test.pro.pkl','rb') as f:
    test_sentences = pickle.load(f)

for i,u in enumerate(test_sentences):
    test_sentences[i] = u[:-5]

In [9]:
for u in test_sentences:
    print(u)

she was taken to ir where 3 peripheral branches of the right renal artery was embolized , and a hd line was placed under sterile procedure in the rij .
pt was successfully extubated , and chest tube removed without complication following resolution of air leak .
encourage/assist with po intake and give supplements at minimum of 3 times daily .
38 year old male with known mitral regurgitation and mitral valve prolapse with new onset atrial fibrillation .
he was afebrile and hd stable at the time of transfer to medicine floor .
patient became anuric with worsening kidney function and fast rise in potassium .
transported to osh where a left chest tube was placed for a pneumothorax and transferred to hospital1 hospital .
the patient underwent emergent catheterization and 2 promus drug eluting stents were placed in the lcx , which was found to be 100 % occluded .
patient hct was stable , did not have hematemesis for 24 hours and was able to tolerat last name ( lf ) , first name3 ( lf ) ocre

In [10]:
import numpy as np
 
def softmax(x, axis=1):
    row_max = x.max(axis=axis)
 
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

tag_values = [0, 1]
tag_values.append("PADT")
tag2idx = {t: i for i, t in enumerate(tag_values)}



In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

test_sts_step2 = []
test_mask_step2 = []

for test_sentence in test_sentences:

    tokenized_sentence = tokenizer.encode(test_sentence)
    input_ids = torch.tensor([tokenized_sentence]).cuda()

    with torch.no_grad():
        output = model(input_ids)
    logits = output[0].to('cpu').numpy()[0]
    prob = softmax(logits)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    new_prob = []
    new_prob_ = []
    for token, label_idx in zip(tokens, label_indices[0]):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_labels.append(tag_values[label_idx])
            new_tokens.append(token)
            new_prob.append(prob[label_idx][1])
            new_prob_.append(prob[label_idx][0])
            
    i = 0
    new_tokens = new_tokens[1:-1]
    new_labels = new_labels[1:-1]
    test_sts_step2.append(new_tokens)
    test_mask_step2.append(new_labels)
#     for token, label in zip(new_tokens, new_labels):
#         print("{:.4f}\t{}\t{}".format(new_prob[i], label, token))
#         i += 1
#     print("")

In [12]:
with open('./data/test_sts_step2_1016.pkl','wb') as f:
    pickle.dump(test_sts_step2,f);

with open('./data/test_mask_step2_1016.pkl','wb') as f:
    pickle.dump(test_mask_step2,f);

    

In [13]:
print(len(test_sts_step2))
print(len(test_mask_step2))

1016
1016
